In [18]:
%load_ext autoreload
%autoreload 2

In [21]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.getcwd()), ".."))
import evaluate
import pandas as pd
import numpy as np
import wandb
import argparse

import torch
from typing import get_args
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from certainty import load_file, CACHE_DIR, seed_everything, RANDOM_SEED, EventType, extract_events
from datasets import Dataset

[autoreload of certainty failed: Traceback (most recent call last):
  File "/home/peder/.pyenv/versions/3.10.13/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/peder/.pyenv/versions/3.10.13/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/peder/.pyenv/versions/3.10.13/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/peder/Skole/certainty/notebooks/../certainty/__init__.py", line 2, in <module>
    from .utils import load_file, convert_events, bootstrap_metrics, id2label, label2id, load_events, logistic_data, create_bootstrap_figures, seed_everythi

ImportError: cannot import name 'extract_events' from 'certainty' (/home/peder/Skole/certainty/notebooks/../certainty/__init__.py)

In [22]:
from certainty import extract_events

ImportError: cannot import name 'extract_events' from 'certainty' (/home/peder/Skole/certainty/notebooks/../certainty/__init__.py)

In [2]:
def get_token_indices(span, offsets):
    start_char, end_char = map(int, span.split(":"))
    start_idx, end_idx = None, None

    for i, (start, end) in enumerate(offsets):
        if start <= start_char < end:  # Find first token in span
            start_idx = i
        if start < end_char <= end:  # Find last token in span
            end_idx = i
            break

    return start_idx, end_idx

In [3]:
def encode_dataset(dataset, tokenizer, label2id):
    encoded = []

    for sample in dataset:
        toks = tokenizer(sample['text'], truncation=True, return_tensors="pt", add_special_tokens=False, return_offsets_mapping=True, padding="max_length", max_length=254)
        offset_mapping = toks.pop("offset_mapping")

        ner = ['O' for _ in range(0, len(toks['input_ids'][0]))]

        for i in range(0, len(sample['events'])):
            event = sample['events'][i]
            trigger_span = event['trigger'][1][0]
            t = event['event_type']
            modality = event['event_modality']

            trigger_start, trigger_end = get_token_indices(trigger_span, offset_mapping[0])

            ner[trigger_start] = "B-" + modality
            for j in range(trigger_start + 1, trigger_end + 1):
                ner[j] = "I-" + modality

        toks['labels'] = torch.tensor([label2id[label] for label in ner])
        toks["input_ids"] = toks["input_ids"].squeeze(0)
        toks["attention_mask"] = toks["attention_mask"].squeeze(0)
        encoded.append(toks)
    return encoded

In [4]:
train = load_file('en_train.json')
train = pd.DataFrame(train).drop_duplicates('text').drop_duplicates('events').to_dict("records")

In [5]:
train[:2]

[{'sent_id': 'bc/CNN_IP_20030329.1600.02/001',
  'text': 'It was in northern Iraq today that an eight artillery round hit the site occupied by Kurdish fighters near Chamchamal',
  'events': [{'event_type': 'Attack',
    'event_polarity': 'Positive',
    'event_genericity': 'Specific',
    'event_modality': 'Asserted',
    'trigger': [['hit'], ['60:63']],
    'arguments': [[['northern Iraq'], ['10:23'], 'Place'],
     [['today'], ['24:29'], 'Time-Within'],
     [['an eight artillery round'], ['35:59'], 'Instrument'],
     [['the site occupied by Kurdish fighters near Chamchamal'],
      ['64:117'],
      'Target']]}]},
 {'sent_id': 'bc/CNN_IP_20030329.1600.02/002',
  'text': 'A day ago it was controlled by Iraqi troops and packed with these land mines but now we can drive far enough to see the outskirts of Kirkuk',
  'events': [{'event_type': 'Transport',
    'event_polarity': 'Positive',
    'event_genericity': 'Specific',
    'event_modality': 'Asserted',
    'trigger': [['drive'], ['

In [6]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased', cache_dir=CACHE_DIR, local_files_only=True, trust_remote_code=True)

In [7]:
label_list = []
id2label = {}
i = 0
id2label[i] = "O"
label_list.append("O")
i += 1
id2label[i] = "B-Asserted"
label_list.append("B-Asserted")
id2label[i + 1] = "I-Asserted"
label_list.append("I-Asserted")
id2label[i + 2] = "B-Other"
label_list.append("B-Other")
id2label[i + 3] = "I-Other"
label_list.append("I-Other")
label2id = {label: idx for idx, label in id2label.items()}


In [8]:
label_list

['O', 'B-Asserted', 'I-Asserted', 'B-Other', 'I-Other']

In [9]:
id2label

{0: 'O', 1: 'B-Asserted', 2: 'I-Asserted', 3: 'B-Other', 4: 'I-Other'}

In [10]:
label2id

{'O': 0, 'B-Asserted': 1, 'I-Asserted': 2, 'B-Other': 3, 'I-Other': 4}

In [12]:
train_encoded = encode_dataset(train, tokenizer, label2id)

In [13]:
train_encoded[2]

{'input_ids': tensor([ 2008,  1005,  1055,  2138,  6056,  4959,  9924, 16405, 29033,  2098,
         2023,  8956,  2597,  2006,  1996,  4564,  2682, 15775, 12458,  3511,
         2389,  1998,  8956,  3629,  2081,  1037, 27151,  7822,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [14]:
model = AutoModelForTokenClassification.from_pretrained(
    'distilbert/distilbert-base-uncased', cache_dir=CACHE_DIR, local_files_only=True, num_labels=len(id2label.items()), trust_remote_code=True, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
train_dataset = Dataset.from_list(train_encoded)

In [16]:
OUTPUT_DIR = "../models/blabla"

training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        evaluation_strategy="epoch",
        save_strategy="no",
        use_cpu=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()



/home/peder/.pyenv/versions/3.10.13/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: peder-august. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

In [75]:
from torch.utils.data import DataLoader, Dataset

In [77]:
train_dataloader = DataLoader(train_encoded, batch_size=16, shuffle=True)

In [78]:
model=model.cpu()

In [80]:
model.eval()
# Storage for all predictions and labels
all_true_labels = []
all_pred_labels = []

# Get predictions
with torch.no_grad():
    for batch in train_dataloader:  # Assuming you're using a DataLoader
        outputs = model(**batch)
        
        # Get logits for token classification
        logits = outputs.logits
        
        # Convert logits to predictions (take the argmax)
        predictions = torch.argmax(logits, dim=-1)
        
        # Get true labels
        labels = batch["labels"]
        
        # Convert to lists and append to storage
        # Only consider tokens that are not padding (attention_mask = 1)
        for i, mask in enumerate(batch["attention_mask"]):
            pred = predictions[i][mask == 1].cpu().tolist()
            true = labels[i][mask == 1].cpu().tolist()
            
            all_pred_labels.extend(pred)
            all_true_labels.extend(true)

In [84]:
len(all_pred_labels)

86082

In [85]:
len(all_true_labels)

86082

In [31]:
def extract_events(labels):
    events = []
    current_event = None

    for idx, label in enumerate(labels):
        if label == 0:  # O tag
            if current_event:
                events.append((current_event[0], idx - 1, current_event[1], current_event[2]))
                current_event = None

        elif label == 1:  # B-Asserted
            if current_event:
                events.append((current_event[0], idx - 1, current_event[1], current_event[2]))
            current_event = (idx, "event", True)  # True for Asserted

        elif label == 2:  # I-Asserted
            if not current_event or current_event[2] != True:
                current_event = (idx, "event", True)

        elif label == 3:  # B-Other
            if current_event:
                events.append((current_event[0], idx - 1, current_event[1], current_event[2]))
            current_event = (idx, "event", False)  # False for Other

        elif label == 4:  # I-Other
            if not current_event or current_event[2] != False:
                current_event = (idx, "event", False)

    if current_event:
        events.append((current_event[0], len(labels) - 1, current_event[1], current_event[2]))

    return events

In [86]:
pred_events = extract_events(all_pred_labels)

In [87]:
true_events = extract_events(all_true_labels)

In [41]:
def match_events(true_events, pred_events):
    discovered = []
    undiscovered = true_events.copy()
    spurious = pred_events.copy()

    for pred_event in pred_events[:]:
        for true_event in true_events[:]:
            if pred_event[0] == true_event[0] and pred_event[1] == true_event[1]:
                discovered.append((true_event, pred_event))
                if true_event in undiscovered:
                    undiscovered.remove(true_event)
                if pred_event in spurious:
                    spurious.remove(pred_event)

    return discovered, undiscovered, spurious

In [89]:
discovered, undiscovered, spurious = match_events(true_events, pred_events)

In [91]:
undiscovered[:10]

[(15, 15, 'event', True),
 (16, 16, 'event', True),
 (58, 58, 'event', False),
 (70, 70, 'event', True),
 (140, 140, 'event', True),
 (174, 175, 'event', True),
 (182, 183, 'event', False),
 (197, 199, 'event', True),
 (230, 232, 'event', False),
 (260, 260, 'event', True)]

In [92]:
spurious[:10]

[(174, 174, 'event', True),
 (175, 175, 'event', True),
 (182, 182, 'event', True),
 (183, 183, 'event', True),
 (278, 278, 'event', True),
 (410, 410, 'event', True),
 (522, 522, 'event', True),
 (530, 530, 'event', True),
 (620, 620, 'event', True),
 (763, 763, 'event', True)]

In [49]:
def calculate_event_metrics(discovered, undiscovered, spurious):
    true_positives = len(discovered)
    false_negatives = len(undiscovered)
    false_positives = len(spurious)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "event_precision": precision,
        "event_recall": recall,
        "event_f1": f1,
        "true_positives": true_positives,
        "false_negatives": false_negatives,
        "false_positives": false_positives
    }

In [93]:
calculate_event_metrics(discovered, undiscovered, spurious)

{'event_precision': 0.7431127012522362,
 'event_recall': 0.4874442619103497,
 'event_f1': 0.588718820861678,
 'true_positives': 2077,
 'false_negatives': 2184,
 'false_positives': 718}

In [51]:
def calculate_factuality_metrics_discovered(discovered, spurious):
    true_asserted = sum(1 for true_event, pred_event in discovered
                        if true_event[3] and pred_event[3])

    true_other = sum(1 for true_event, pred_event in discovered
                     if not true_event[3] and not pred_event[3])

    false_asserted = sum(1 for true_event, pred_event in discovered
                         if not true_event[3] and pred_event[3])

    false_other = sum(1 for true_event, pred_event in discovered
                      if true_event[3] and not pred_event[3])

    asserted_precision = true_asserted / (true_asserted + false_asserted) if (true_asserted + false_asserted) > 0 else 0
    asserted_recall = true_asserted / (true_asserted + false_other) if (true_asserted + false_other) > 0 else 0
    asserted_f1 = 2 * asserted_precision * asserted_recall / (asserted_precision + asserted_recall) if (asserted_precision + asserted_recall) > 0 else 0

    other_precision = true_other / (true_other + false_other) if (true_other + false_other) > 0 else 0
    other_recall = true_other / (true_other + false_asserted) if (true_other + false_asserted) > 0 else 0
    other_f1 = 2 * other_precision * other_recall / (other_precision + other_recall) if (other_precision + other_recall) > 0 else 0

    return {
        "discovered_true_asserted": true_asserted,
        "discovered_true_other": true_other,
        "discovered_false_asserted": false_asserted,
        "discovered_false_other": false_other,
        "discovered_asserted_precision": asserted_precision,
        "discovered_asserted_recall": asserted_recall,
        "discovered_asserted_f1": asserted_f1,
        "discovered_other_precision": other_precision,
        "discovered_other_recall": other_recall,
        "discovered_other_f1": other_f1
    }

In [94]:
calculate_factuality_metrics_discovered(discovered, spurious)

{'discovered_true_asserted': 1693,
 'discovered_true_other': 0,
 'discovered_false_asserted': 384,
 'discovered_false_other': 0,
 'discovered_asserted_precision': 0.8151179585941262,
 'discovered_asserted_recall': 1.0,
 'discovered_asserted_f1': 0.8981432360742706,
 'discovered_other_precision': 0,
 'discovered_other_recall': 0.0,
 'discovered_other_f1': 0}

In [53]:

def calculate_factuality_metrics_all(discovered, undiscovered, spurious):
    true_asserted = sum(1 for true_event, pred_event in discovered
                        if true_event[3] and pred_event[3])

    true_other = sum(1 for true_event, pred_event in discovered
                     if not true_event[3] and not pred_event[3])

    false_asserted = sum(1 for true_event, pred_event in discovered
                         if not true_event[3] and pred_event[3])

    false_other = sum(1 for true_event, pred_event in discovered
                      if true_event[3] and not pred_event[3])

    false_negative_asserted = sum(1 for event in undiscovered if event[3])  # Asserted events that weren't discovered
    false_negative_other = sum(1 for event in undiscovered if not event[3])  # Other events that weren't discovered

    false_positive_asserted = sum(1 for event in spurious if event[3])  # Spurious events predicted as Asserted
    false_positive_other = sum(1 for event in spurious if not event[3])  # Spurious events predicted as Other

    all_tp_asserted = true_asserted
    all_tp_other = true_other
    all_fp_asserted = false_asserted + false_positive_asserted
    all_fp_other = false_other + false_positive_other
    all_fn_asserted = false_other + false_negative_asserted
    all_fn_other = false_asserted + false_negative_other

    asserted_precision = all_tp_asserted / (all_tp_asserted + all_fp_asserted) if (all_tp_asserted + all_fp_asserted) > 0 else 0
    asserted_recall = all_tp_asserted / (all_tp_asserted + all_fn_asserted) if (all_tp_asserted + all_fn_asserted) > 0 else 0
    asserted_f1 = 2 * asserted_precision * asserted_recall / (asserted_precision + asserted_recall) if (asserted_precision + asserted_recall) > 0 else 0

    other_precision = all_tp_other / (all_tp_other + all_fp_other) if (all_tp_other + all_fp_other) > 0 else 0
    other_recall = all_tp_other / (all_tp_other + all_fn_other) if (all_tp_other + all_fn_other) > 0 else 0
    other_f1 = 2 * other_precision * other_recall / (other_precision + other_recall) if (other_precision + other_recall) > 0 else 0

    return {
        "all_true_positive_asserted": all_tp_asserted,
        "all_true_positive_other": all_tp_other,
        "all_false_positive_asserted": all_fp_asserted,
        "all_false_positive_other": all_fp_other,
        "all_false_negative_asserted": all_fn_asserted,
        "all_false_negative_other": all_fn_other,
        "all_asserted_precision": asserted_precision,
        "all_asserted_recall": asserted_recall,
        "all_asserted_f1": asserted_f1,
        "all_other_precision": other_precision,
        "all_other_recall": other_recall,
        "all_other_f1": other_f1
    }

In [95]:
calculate_factuality_metrics_all(discovered, undiscovered, spurious)

{'all_true_positive_asserted': 1693,
 'all_true_positive_other': 0,
 'all_false_positive_asserted': 1102,
 'all_false_positive_other': 0,
 'all_false_negative_asserted': 1523,
 'all_false_negative_other': 1045,
 'all_asserted_precision': 0.6057245080500895,
 'all_asserted_recall': 0.5264303482587065,
 'all_asserted_f1': 0.56330061553818,
 'all_other_precision': 0,
 'all_other_recall': 0.0,
 'all_other_f1': 0}